In [28]:
with open("seed.txt", "r") as f:
    seed = int(f.read().strip())

print(f"seed = {seed}")

seed = 52772


In [29]:
#Task 1: Regression — Regularization on Housing Data
#1.1 1A. Data Preprocessing (Required)
#1. Use your seed to create a 60/20/20 train/validation/test split.

#load dataset
import urllib.request
import pandas as pd
import numpy as np

url = "https://lib.stat.cmu.edu/datasets/boston"
response = urllib.request.urlopen(url)
raw = response.read().decode()
lines = raw.strip().splitlines()
data_lines = lines[22:] #dataset starts from line 23 

dataset = []

#each data contains 2 lines
for i in range(0, len(data_lines), 2):
    line1 = data_lines[i].strip().split()
    line2 = data_lines[i+1].strip().split()
    dataset.append(line1 + line2)

column_names = [
    "CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM",
    "AGE", "DIS", "RAD", "TAX", "PTRATIO", "B", "LSTAT", "MEDV"
]

df = pd.DataFrame(dataset, columns=column_names).astype(float)

#display(df)

total_rows = len(df)
print(f"Total rows in dataset: {total_rows}")

np.random.seed(seed)
indices = np.random.permutation(total_rows)

train_end = int(0.6 * total_rows)
validation_end = int(0.8 * total_rows)

train_index = indices[:train_end] #60%
validation_index   = indices[train_end:validation_end] #20%
test_index  = indices[validation_end:] #20%

train_df = df.iloc[train_index]
validation_df   = df.iloc[validation_index]
test_df  = df.iloc[test_index]

#verify split correctly
print(f"Train set: {len(train_df)} rows,", f"Validation set: {len(validation_df)} rows,", f"Test set: {len(test_df)}")
print(f"Train: {len(train_df)/len(df)*100:.1f}%, ", f"Validation: {len(validation_df)/len(df)*100:.1f}%, ", f"Test: {len(test_df)/len(df)*100:.1f}%")


Total rows in dataset: 506
Train set: 303 rows, Validation set: 101 rows, Test set: 102
Train: 59.9%,  Validation: 20.0%,  Test: 20.2%


In [30]:
#2. Handle missing values (if any) and apply appropriate transformations:
#• If categorical columns exist: apply one-hot encoding.
#• Create two interaction features of your choice; justify why they may help.

df.isnull().sum()
#So there is no missing values in this dataset

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
MEDV       0
dtype: int64